In [ ]:
#掩码语言训练

In [1]:
#导入相关包
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForLanguageModeling,Trainer,TrainingArguments

In [2]:
#加载数据集
ds = Dataset.load_from_disk("./wiki_cn_filtered")

In [3]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

In [4]:
#数据处理
tokenizer = AutoTokenizer.from_pretrained("D:/Hugging Face Hub/bloom-389m-zh")
def process_fun(examples):
    content =[i+tokenizer.eos_token for i in examples["completion"]]
    return tokenizer(content,max_length = 384,truncation =True)

In [5]:
tokenizer_ds = ds.map(process_fun,batched=True,remove_columns=ds.column_names)
print(tokenizer_ds[0])

{'input_ids': [13110, 34800, 13535, 916, 33156, 10, 256, 576, 387, 479, 681, 5453, 10955, 915, 24124, 5317, 13110, 6573, 20757, 13535, 355, 5358, 1490, 583, 28056, 1407, 3855, 671, 6113, 189, 6732, 4302, 9488, 3434, 6900, 1322, 355, 37336, 9825, 4608, 13461, 1359, 5358, 355, 5317, 13110, 34800, 4433, 7189, 25722, 29747, 13110, 1498, 12047, 6347, 23563, 2139, 2066, 420, 29288, 25, 15, 7635, 39288, 355, 1484, 5835, 6272, 23, 15, 4180, 39288, 355, 5358, 4516, 11621, 23, 15, 10641, 4887, 1712, 420, 2450, 31163, 8085, 11621, 5358, 553, 9888, 2731, 21335, 5358, 553, 9876, 14011, 4434, 5358, 553, 21484, 4514, 17170, 25871, 8085, 5358, 553, 17489, 6945, 11097, 13535, 641, 33623, 1484, 5835, 1689, 2063, 5358, 569, 5835, 671, 16225, 3422, 189, 13, 23158, 27894, 33227, 1022, 11396, 3347, 1813, 1504, 6566, 1813, 355, 9155, 8633, 1504, 2063, 1813, 189, 13, 23158, 813, 7817, 5358, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(tokenizer_ds,batch_size=2,collate_fn=DataCollatorForLanguageModeling(tokenizer,mlm=False))

In [ ]:
next(enumerate(dl))

In [ ]:
#创建模型
model = AutoModelForCausalLM.from_pretrained("D:/Hugging Face Hub/bloom-389m-zh")

In [ ]:
#不配置评估函数

In [ ]:
args = TrainingArguments(
    output_dir="./causa",
    per_device_train_batch_size = 1,
    logging_steps = 100,
    num_train_epochs=1
)

In [ ]:
trainer = Trainer(
    args = args,
    model = model,
    train_dataset=tokenizer_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm = False)
)

In [ ]:
#训练
trainer.train()

In [ ]:
#模型推理
from transformers import pipeline
pipe = pipeline("text-generation",model=model,tokenizer=tokenizer,device=0)

In [ ]:
pipe("西安交通学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大",max_length = 128,do_sample=True)